# Planejamento da solução

#### Perguntas de negócio

1. Qual a taxa atual de Churn da TopBank? Como ela varia mensalmente?
2. Qual a Performance do modelo em classificar os clientes como churns?
3. Qual o retorno esperado, em termos de faturamento, se a empresa utilizar seu modelo para evitar o churn dos clientes?



####  Tasks

1. Ler os dados
2. Estudar os dados
3. Limpar os dados
4. Criar features
5. Selecionar as features mais importantes
6. Testar modelos de classificação
7. Melhorar os parâmetros
8. Fazer cross validation
9. Analisar performance do modelo
10. Traduzir o erro em resultado de negócio

#### Saída

1. Tabela de teste com clientes classificados se estão ou não em churn
2. Deploy no Heroku
3. API no Google Sheets
4. Relatório respondendo perguntas de negócio
5. Principais hipóteses da EDA em Tableau (talvez)



# 0.0. Imports

In [1]:
import pandas         as pd
import numpy          as np
import seaborn        as sns
import plotly.express as px


from matplotlib import pyplot as plt



# 0.1. Helper Functions

In [2]:
def num_attributes(df1):
    
    num_attributes = df1.select_dtypes(['int64', 'float64'])

    #central tendency
    ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
    ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

    #dispersion
    d1 = pd.DataFrame(num_attributes.apply(np.min)).T
    d2 = pd.DataFrame(num_attributes.apply(np.max)).T
    d3 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
    d4 = pd.DataFrame(num_attributes.apply(np.std)).T
    d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

    metrics = pd.concat( [d1, d2, d3, ct1, ct2, d4, d5, d6] ).T.reset_index()
    metrics.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std','skew', 'kurtosis']
    return metrics

def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'ggplot')
    plt.rcParams['figure.figsize'] = [24, 9]
    plt.rcParams['font.size'] = 24
    
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set(font_scale = 3)
    sns.set()
    
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


## 0.2. Load Data

In [3]:
df_raw = pd.read_csv('../data/churn.csv')

# 1.0. Data Description

## 1.1. Rename Columns

In [4]:
df1=df_raw.copy()

df1.columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited']
df1.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 1.2. Data Shape

In [5]:
print("Number of rows: {}".format(df1.shape[0]))
print("Number of columns: {}".format(df1.shape[1]))

Number of rows: 10000
Number of columns: 14


## 1.3. Data Types

In [6]:
df1.dtypes

row_number            int64
customer_id           int64
surname              object
credit_score          int64
geography            object
gender               object
age                   int64
tenure                int64
balance             float64
num_products          int64
has_cr_card           int64
is_active_member      int64
estimated_salary    float64
exited                int64
dtype: object

## 1.4. Check NA

In [7]:
df1.isna().sum()

row_number          0
customer_id         0
surname             0
credit_score        0
geography           0
gender              0
age                 0
tenure              0
balance             0
num_products        0
has_cr_card         0
is_active_member    0
estimated_salary    0
exited              0
dtype: int64

## 1.5. FIll NA


No NA to replace

## 1.5. Descriptive Analysis

### 1.5.1. Num Attributes

In [8]:
metrics = num_attributes(df1)

#drop binary features
metrics = metrics.drop([0,1,7,8,10], axis=0)
metrics

,attributes,min,max,range,mean,median,std,skew,kurtosis
2,credit_score,350.00,850.00,500.00,650.528800,652.000,96.648466,-0.071607,-0.425726
3,age,18.00,92.00,74.00,38.921800,37.000,10.487282,1.011320,1.395347
4,tenure,0.00,10.00,10.00,5.012800,5.000,2.892030,0.010991,-1.165225
5,balance,0.00,250898.09,250898.09,76485.889288,97198.540,62394.285254,-0.141109,-1.489412
6,num_products,1.00,4.00,3.00,1.530200,1.000,0.581625,0.745568,0.582981
9,estimated_salary,11.58,199992.48,199980.90,100090.239881,100193.915,57507.617221,0.002085,-1.181518


### 1.5.2. Categorical Attributes

In [9]:
cat_attributes = df1.drop(columns=['row_number', 'customer_id','credit_score', 'age', 'tenure', 'balance', 'num_products', 'estimated_salary'], axis=1)
cat_attributes.head()

,surname,geography,gender,has_cr_card,is_active_member,exited
0,Hargrave,France,Female,1,1,1
1,Hill,Spain,Female,0,1,0
2,Onio,France,Female,1,0,1
3,Boni,France,Female,0,0,0
4,Mitchell,Spain,Female,1,1,0


#### Surname

In [10]:
# cat_attributes['surname'].value_counts()

#### Gender

In [29]:
cat_attributes['gender'].value_counts(normalize=True)

Male      0.5457
Female    0.4543
Name: gender, dtype: float64

#### Geography

In [28]:
cat_attributes['geography'].value_counts(normalize=True)

France     0.5014
Germany    0.2509
Spain      0.2477
Name: geography, dtype: float64

#### Credit Card

In [27]:
cat_attributes['has_cr_card'].value_counts(normalize=True)

1    0.7055
0    0.2945
Name: has_cr_card, dtype: float64

#### Active member

In [25]:
cat_attributes['is_active_member'].value_counts(normalize=True)

1    0.5151
0    0.4849
Name: is_active_member, dtype: float64

#### Exited (target)

In [26]:
cat_attributes['exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: exited, dtype: float64

# 2.0. Feature Engineering

In [67]:
df2 = df1.copy()


#drop row number
df2 = df2.drop('row_number', axis=1)

#salary / balance

df2.loc[:,'balance_salary'] = df2.loc[:,'balance']/df2.loc[:,'estimated_salary']




# 3.0. Exploratory Data Analysis

In [68]:
df3 = df2.copy()

## 3.1. Unavariate Analisys

In [69]:
df_yes = df3[df3['exited'] == 1]
df_no = df3[df3['exited'] == 0]

df3.columns

Index(['customer_id', 'surname', 'credit_score', 'geography', 'gender', 'age',
       'tenure', 'balance', 'num_products', 'has_cr_card', 'is_active_member',
       'estimated_salary', 'exited', 'balance_salary'],
      dtype='object')

### Age

In [70]:
fig = px.box(df3, x="exited", y="age")
fig.show()

In [64]:
# ditribution yes by age}
fig = px.histogram(df_yes, x="age")
fig.show()


In [63]:
# ditribution no by age
fig = px.histogram(df_no, x="age")
fig.show()


In [50]:
# aux = df3[['customer_id','age', 'exited']].groupby(['age', 'exited']).count().rename(columns={'customer_id': 'n_customers'}).reset_index()

# fig = px.histogram(aux, x="age", y="n_customers", color="exited", marginal="rug",
#                    hover_data=aux.columns, nbins=30)
# fig.show()


The target feature is concentrated in middle age [40-50].

### Geography

In [51]:
aux = df3[['customer_id','geography', 'exited']].groupby(['geography', 'exited']).count().rename(columns={'customer_id': 'n_customers'}).reset_index()

fig = px.bar(aux, x="geography", y="n_customers", color="exited", title="Geography Distribution")
fig.show()



### Gender

In [52]:
aux = df3[['customer_id','gender', 'exited']].groupby(['gender', 'exited']).count().rename(columns={'customer_id': 'n_customers'}).reset_index()


fig = px.bar(aux, x="gender", y="n_customers", color="exited", title="Gender Distribution")
fig.show()


### Tenure

In [53]:
fig = px.box(df3, x="exited", y="tenure")
fig.show()


### Balance

In [54]:
fig = px.box(df3, x="exited", y="balance")
fig.show()


In [61]:
fig = px.histogram(df_yes, x="balance")
fig.show()

In [62]:
fig = px.histogram(df_no, x="balance")
fig.show()

Distribution is similar, but the amount is much larger in "no exit"

### Salary Balance

In [56]:
fig = px.box(df3, x="exited", y="salary_balance")
fig.show()

### Credit card

In [57]:
aux = df3[['customer_id','has_cr_card', 'exited']].groupby(['has_cr_card', 'exited']).count().rename(columns={'customer_id': 'n_customers'}).reset_index()

fig = px.bar(aux, x="has_cr_card", y="n_customers", color="exited", title="Credit Card Distribution")
fig.show()

A lot more has exited with credit card. More than 2 times the number. 

### Estimated Salary

In [58]:
fig = px.box(df3, x='exited', y='estimated_salary')
fig.show()

In [59]:
fig = px.histogram(df_yes, x="estimated_salary")
fig.show()

In [60]:
fig = px.histogram(df_no, x="estimated_salary")
fig.show()

There are low correlation to the target, but we could analyse this combined with another feature.

1. Gender
1. Age

## 3.2. Bivariate Analysis

How those features impact on the target feature

### Age & Gender 

### Age & Balance

### Age & Balance/Salary